In [3]:
import pandas as pd
import numpy as np
import torch
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DTYPE = torch.float64


init_sigmasq   = 13.059
init_range_lat = 0.154 
init_range_lon = 0.195
init_advec_lat = 0.0218
init_range_time = 0.7
init_advec_lon = -0.1689
init_nugget    = 0.247


# True Parameters Vector
init_phi2 = 1.0 / init_range_lon
init_phi1 = init_sigmasq * init_phi2
init_phi3 = (init_range_lon / init_range_lat)**2
init_phi4 = (init_range_lon / init_range_time)**2

true_params_vec = torch.tensor([
    np.log(init_phi1), np.log(init_phi2), np.log(init_phi3), np.log(init_phi4),
    init_advec_lat, init_advec_lon, np.log(init_nugget)
], device=DEVICE, dtype=DTYPE)


true_params = torch.tensor([init_sigmasq, init_range_lat, init_range_lon,
                            init_range_time, init_advec_lat, init_advec_lon,
                            init_nugget], device=DEVICE, dtype=DTYPE)

compare heads 0 vs 100 vs 300

In [2]:
df0 = pd.read_csv("/Users/joonwonlee/Documents/GEMS_TCO-1/outputs/day/sim_vecc_0heads_1217_v050_LBFGS_18126.0.csv")
df100 = pd.read_csv("/Users/joonwonlee/Documents/GEMS_TCO-1/outputs/day/sim_vecc_100heads_1217_v050_LBFGS_18126.0.csv")
df300 = pd.read_csv("/Users/joonwonlee/Documents/GEMS_TCO-1/outputs/day/sim_vecc_300heads_1217_v050_LBFGS_18126.0.csv")
df400 = pd.read_csv("/Users/joonwonlee/Documents/GEMS_TCO-1/outputs/day/sim_vecc_400heads_1221_v050_LBFGS_18126.0.csv")
df500 = pd.read_csv("/Users/joonwonlee/Documents/GEMS_TCO-1/outputs/day/sim_vecc_500heads_1221_v050_LBFGS_18126.0.csv")


print(df0['loss'].mean(), df100['loss'].mean(), df300['loss'].mean(), df400['loss'].mean(), df500['loss'].mean())

1.2435295983067052 1.24374378744264 1.2438515511938197 1.2434965229754698 1.243455522781668


In [4]:
df0 = pd.read_csv("/Users/joonwonlee/Documents/GEMS_TCO-1/outputs/day/sim_vecc_0heads_1217_v050_LBFGS_18126.0.csv")
df100 = pd.read_csv("/Users/joonwonlee/Documents/GEMS_TCO-1/outputs/day/sim_vecc_100heads_1217_v050_LBFGS_18126.0.csv")
df300 = pd.read_csv("/Users/joonwonlee/Documents/GEMS_TCO-1/outputs/day/sim_vecc_300heads_1217_v050_LBFGS_18126.0.csv")
df400 = pd.read_csv("/Users/joonwonlee/Documents/GEMS_TCO-1/outputs/day/sim_vecc_400heads_1221_v050_LBFGS_18126.0.csv")
df500 = pd.read_csv("/Users/joonwonlee/Documents/GEMS_TCO-1/outputs/day/sim_vecc_500heads_1221_v050_LBFGS_18126.0.csv")

df0 = df0.iloc[:,5:(5+7)]
df100 = df100.iloc[:,5:(5+7)]
df300 = df300.iloc[:,5:(5+7)]
df400 = df400.iloc[:,5:(5+7)]
df500 = df500.iloc[:,5:(5+7)]

# mape

import numpy as np

rmse_list = []

for i in range(7):
    # Get the true value for the current parameter
    true_val = true_params_vec[i].cpu().numpy()
    
    # Calculate RMSE for each scenario
    # Formula: sqrt( mean( (estimate - truth)^2 ) )
    rmse0 = np.sqrt(np.mean((df0.iloc[:,i] - true_val)**2))
    rmse100 = np.sqrt(np.mean((df100.iloc[:,i] - true_val)**2))
    rmse300 = np.sqrt(np.mean((df300.iloc[:,i] - true_val)**2))
    rmse400 = np.sqrt(np.mean((df400.iloc[:,i] - true_val)**2))
    rmse500 = np.sqrt(np.mean((df500.iloc[:,i] - true_val)**2))
    
    # Optional: Store for later use
    rmse_list.append([rmse0, rmse100, rmse300, rmse400, rmse500])
    
    print(f'Parameter {i+1}: RMSE 0 heads: {rmse0:.4f}, RMSE 100 heads: {rmse100:.4f}, RMSE 300 heads: {rmse300:.4f}, RMSE 400 heads: {rmse400:.4f}, RMSE 500 heads: {rmse500:.4f}')


Parameter 1: RMSE 0 heads: 0.0546, RMSE 100 heads: 0.0468, RMSE 300 heads: 0.0322, RMSE 400 heads: 0.0287, RMSE 500 heads: 0.0254
Parameter 2: RMSE 0 heads: 0.1233, RMSE 100 heads: 0.0972, RMSE 300 heads: 0.0547, RMSE 400 heads: 0.0395, RMSE 500 heads: 0.0317
Parameter 3: RMSE 0 heads: 0.0161, RMSE 100 heads: 0.0156, RMSE 300 heads: 0.0145, RMSE 400 heads: 0.0139, RMSE 500 heads: 0.0137
Parameter 4: RMSE 0 heads: 0.3337, RMSE 100 heads: 0.3468, RMSE 300 heads: 0.3482, RMSE 400 heads: 0.3368, RMSE 500 heads: 0.3190
Parameter 5: RMSE 0 heads: 0.0055, RMSE 100 heads: 0.0055, RMSE 300 heads: 0.0055, RMSE 400 heads: 0.0048, RMSE 500 heads: 0.0048
Parameter 6: RMSE 0 heads: 0.0091, RMSE 100 heads: 0.0089, RMSE 300 heads: 0.0079, RMSE 400 heads: 0.0088, RMSE 500 heads: 0.0076
Parameter 7: RMSE 0 heads: 0.9404, RMSE 100 heads: 0.7565, RMSE 300 heads: 0.4868, RMSE 400 heads: 0.4527, RMSE 500 heads: 0.4028


In [5]:
import pandas as pd

# 데이터프레임 리스트와 컬럼명 정의
dfs = [df0, df100, df300, df400, df500]
column_names = ['0 heads', '100 heads', '300 heads', '400 heads', '500 heads']
index_names = [f'Parameter {i+1}' for i in range(7)]

# 각 파라미터별(행) 시나리오별(열) STD 계산
std_data = []

for i in range(7):
    # 각 df의 i번째 컬럼에 대해 std 계산
    row_stds = [df.iloc[:, i].std() for df in dfs]
    std_data.append(row_stds)

# 결과를 데이터프레임으로 변환
df_std = pd.DataFrame(std_data, columns=column_names, index=index_names)

print("=== Standard Deviation (STD) Table ===")
print(df_std)

# (선택 사항) 보기 좋게 소수점 포맷팅을 원하시면 아래 주석을 해제하세요
# print(df_std.round(4))

=== Standard Deviation (STD) Table ===
              0 heads  100 heads  300 heads  400 heads  500 heads
Parameter 1  0.015102   0.015192   0.014060   0.013649   0.013506
Parameter 2  0.039441   0.041196   0.039605   0.030445   0.029480
Parameter 3  0.011914   0.011941   0.011910   0.011839   0.011867
Parameter 4  0.068204   0.067811   0.065894   0.054595   0.051867
Parameter 5  0.005549   0.005531   0.005516   0.004806   0.004804
Parameter 6  0.005588   0.005578   0.005144   0.006074   0.005936
Parameter 7  0.363741   0.295666   0.210278   0.210986   0.198332


# compare vecc vs dw

In [5]:
dw_re = pd.read_csv("/Users/joonwonlee/Documents/GEMS_TCO-1/outputs/day/sim_dw_vecc_mm16_h300_122425/sim_reg_dW_v05_18126.csv")
vecc_re = pd.read_csv("/Users/joonwonlee/Documents/GEMS_TCO-1/outputs/day/sim_dw_vecc_mm16_h300_122425/sim_reg_vecc_v05_18126.csv")

# 12 21 25: vecchia used 300 heads mm16 and LOC_ERR_STD = 0.02

dw_irre = pd.read_csv("/Users/joonwonlee/Documents/GEMS_TCO-1/outputs/day/sim_dw_vecc_mm16_h300_122425/sim_irre_dW_v05_18126.csv")
vecc_irre = pd.read_csv("/Users/joonwonlee/Documents/GEMS_TCO-1/outputs/day/sim_dw_vecc_mm16_h300_122425/sim_irre_vecc_v05_18126.csv")


dw_re = dw_re.iloc[:,4:(4+7)]
dw_irre = dw_irre.iloc[:,4:(4+7)]
vecc_re = vecc_re.iloc[:,4:(4+7)]
vecc_irre = vecc_irre.iloc[:,4:(4+7)]



In [7]:
import numpy as np
import pandas as pd

results = []

for i in range(7):
    # 참값 (Scalar)
    true_val = true_params[i].cpu().numpy()
    
    # 참값이 0에 너무 가까우면 나눗셈 에러가 날 수 있으므로 안전장치 추가 (선택 사항)
    # 여기서는 파라미터들이 0이 아니라고 가정하고 진행합니다.
    
    # --- REGULAR GRID (RE) ---
    # 1. Absolute Metrics
    bias_dw_re = np.mean(dw_re.iloc[:,i] - true_val)
    bias_vecc_re = np.mean(vecc_re.iloc[:,i] - true_val)
    rmse_dw_re = np.sqrt(np.mean((dw_re.iloc[:,i] - true_val)**2))
    rmse_vecc_re = np.sqrt(np.mean((vecc_re.iloc[:,i] - true_val)**2))
    
    # 2. Relative Metrics (%)
    # Relative Bias = Mean( (Est - True) / True )
    rel_bias_dw_re = np.mean((dw_re.iloc[:,i] - true_val) / true_val)
    rel_bias_vecc_re = np.mean((vecc_re.iloc[:,i] - true_val) / true_val)
    
    # RMSRE = Sqrt( Mean( ((Est - True) / True)^2 ) )
    rmsre_dw_re = np.sqrt(np.mean(((dw_re.iloc[:,i] - true_val) / true_val)**2))
    rmsre_vecc_re = np.sqrt(np.mean(((vecc_re.iloc[:,i] - true_val) / true_val)**2))

    # --- IRREGULAR GRID (IRRE) ---
    # 1. Absolute Metrics
    bias_dw_irre = np.mean(dw_irre.iloc[:,i] - true_val)
    bias_vecc_irre = np.mean(vecc_irre.iloc[:,i] - true_val)
    rmse_dw_irre = np.sqrt(np.mean((dw_irre.iloc[:,i] - true_val)**2))
    rmse_vecc_irre = np.sqrt(np.mean((vecc_irre.iloc[:,i] - true_val)**2))
    
    # 2. Relative Metrics (%)
    rel_bias_dw_irre = np.mean((dw_irre.iloc[:,i] - true_val) / true_val)
    rel_bias_vecc_irre = np.mean((vecc_irre.iloc[:,i] - true_val) / true_val)
    
    rmsre_dw_irre = np.sqrt(np.mean(((dw_irre.iloc[:,i] - true_val) / true_val)**2))
    rmsre_vecc_irre = np.sqrt(np.mean(((vecc_irre.iloc[:,i] - true_val) / true_val)**2))

    results.append({
        "Param": f"P{i+1}",
        "True Value": round(float(true_val), 4), # 참값도 같이 보면 해석이 더 쉽습니다.
        
        # --- Regular ---
        "DW_Re RelBias": rel_bias_dw_re,
        "Vecc_Re RelBias": rel_bias_vecc_re,
        "DW_Re RMSRE": rmsre_dw_re,
        "Vecc_Re RMSRE": rmsre_vecc_re,
        
        # --- Irregular ---
        "DW_Irre RelBias": rel_bias_dw_irre,
        "Vecc_Irre RelBias": rel_bias_vecc_irre,
        "DW_Irre RMSRE": rmsre_dw_irre,
        "Vecc_Irre RMSRE": rmsre_vecc_irre,
    })

# Create DataFrame
df_rel_results = pd.DataFrame(results)

# Display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# 보기 편하게 퍼센트로 해석하기 위해 출력
print("=== Relative Metrics (RelBias, RMSRE) ===")
print(df_rel_results.round(4))

=== Relative Metrics (RelBias, RMSRE) ===
  Param  True Value  DW_Re RelBias  Vecc_Re RelBias  DW_Re RMSRE  Vecc_Re RMSRE  DW_Irre RelBias  Vecc_Irre RelBias  DW_Irre RMSRE  Vecc_Irre RMSRE
0    P1     13.0590         0.0047          -0.0335       0.0321         0.0377          -0.0627            -0.0973         0.0726           0.0993
1    P2      0.1540         0.0056          -0.0474       0.0429         0.0535           0.2281             0.1599         0.2357           0.1636
2    P3      0.1950         0.0044          -0.0483       0.0420         0.0540           0.0503            -0.0088         0.0734           0.0321
3    P4      0.7000         0.0056          -0.0063       0.0595         0.0446           0.5481             0.5587         0.5545           0.5793
4    P5      0.0218         0.1365          -0.0748       1.0007         0.4590           0.9419             0.9184         1.0165           1.2483
5    P6     -0.1689        -0.0134           0.0578       0.1283      

for perfect regular grid, dw better in terms of bias for p1 but for p3,p4,p5,p6 vecc is more efficient by looking at RMSE. Vecc underestimate sigma^2 (p1)

for irregular grid(then gridded) 

Two models both overestimate nugget and underestimate sigma^2
and RMSE for vecc spike for p6 which is weird because in real data vecc better capture longitude wind direction

In [8]:
print(f'dw_re_std: \n{dw_re.std()}, {np.mean(dw_re.std())}')
print(f'vecc_re_std: \n{vecc_re.std()}, {np.mean(vecc_re.std())}')
print(f'dw_irre_std: \n{dw_irre.std()}, {np.mean(dw_irre.std())}')
print(f'vecc_irre_std: \n{vecc_irre.std()}, {np.mean(vecc_irre.std())}')

dw_re_std: 
sigma         0.416274
range_lat     0.006588
range_lon     0.008182
range_time    0.041653
advec_lat     0.021719
advec_lon     0.021652
nugget        0.046690
dtype: float64, 0.08039405341084722
vecc_re_std: 
sigma         0.229282
range_lat     0.003847
range_lon     0.004721
range_time    0.031069
advec_lat     0.009921
advec_lon     0.010833
nugget        0.031440
dtype: float64, 0.045873226214196934
dw_irre_std: 
sigma         0.479942
range_lat     0.009205
range_lon     0.010471
range_time    0.059150
advec_lat     0.008373
advec_lon     0.009566
nugget        0.052341
dtype: float64, 0.089864188839735
vecc_irre_std: 
sigma         0.262477
range_lat     0.005385
range_lon     0.006042
range_time    0.107640
advec_lat     0.018520
advec_lon     0.176874
nugget        0.029622
dtype: float64, 0.0866516040834151


Final Interpretable Params: {'sigma_sq': 12.334784480583942, 'range_lon': 0.16237317690668224, 'range_lat': 0.152229972203287, 'range_time': 1.0681601950135797, 'advec_lat': 0.017573239903560148, 'advec_lon': -0.16538909174735658, 'nugget': 0.27052157933315374}


Final Interpretable Params: {'sigma_sq': 12.379046683462734, 'range_lon': 0.16199143920537762, 'range_lat': 0.15222849133531485, 'range_time': 1.0550803727229192, 'advec_lat': 0.02179873630530265, 'advec_lon': -0.17814740293659534, 'nugget': 0.26146881322799814}


Final Parameters (Natural Scale): sigmasq: 12.8994, range_lat: 0.1702, range_lon: 0.1788, range_time: 0.9948, advec_lat: 0.0138, advec_lon: -0.1643, nugget: 0.4666


real data

In [34]:

dw_real = pd.read_csv("/Users/joonwonlee/Documents/GEMS_TCO-1/outputs/day/real_dw_vecc_estimates_122425/real_dw_18126.csv")
vecc_real = pd.read_csv("/Users/joonwonlee/Documents/GEMS_TCO-1/outputs/day/real_dw_vecc_estimates_122425/real_vecc_h1000_mm16_18126.csv")


dw_real = dw_real.iloc[:28,4:(4+7)]
vecc_real = vecc_real.iloc[:28,4:(4+7)]

print(f'dw_real_std: \n{dw_real.std()}, {np.mean(dw_real.std())}')
print(f'vecc_real_std: \n{vecc_real.std()}, {np.mean(vecc_real.std())}')

dw_real_std: 
sigma         2.971631
range_lat     0.040695
range_lon     0.061084
range_time    0.292563
advec_lat     0.069809
advec_lon     0.137927
nugget        0.745278
dtype: float64, 0.6169981042921118
vecc_real_std: 
sigma         2.338683
range_lat     0.048754
range_lon     0.067624
range_time    0.321468
advec_lat     1.656713
advec_lon     1.110304
nugget        0.702045
dtype: float64, 0.8922272012824829


debiased whittle fails to capture longituve wind direction on day 2, 9, 13 and 22

In [38]:
dw_real

,sigma,range_lat,range_lon,range_time,advec_lat,advec_lon,nugget
0,9.4033,0.1261,0.1428,1.0413,0.0146,-0.2040,0.4246
1,6.9982,0.1400,0.1830,1.3232,-0.0147,0.2415,2.4503
2,17.4925,0.1538,0.1915,1.6396,-0.0530,-0.2803,1.5814
3,12.6267,0.1589,0.2141,1.5550,0.0337,-0.1499,1.1794
4,15.2663,0.1620,0.2176,1.5505,0.0875,-0.3102,0.2564
5,8.0439,0.1314,0.2004,1.0993,-0.0000,-0.0000,2.8706
6,10.2491,0.1278,0.2154,0.9205,-0.0335,-0.1025,1.4607
7,11.0829,0.1617,0.1666,0.7910,-0.0195,-0.2146,0.6078
8,9.0215,0.1488,0.1967,1.0251,0.0226,-0.2038,0.5579
9,4.1499,0.1244,0.2215,0.9538,-0.0083,0.1150,0.6908


In [37]:
vecc_real

,sigma,range_lat,range_lon,range_time,advec_lat,advec_lon,nugget
0,12.7072,0.1485,0.1906,1.1627,0.0245,-0.1766,0.2602
1,12.5313,0.2188,0.3123,1.9063,-0.0125,-0.2240,2.1680
2,15.9515,0.1608,0.2089,1.5086,-0.0417,-0.2536,1.4950
3,12.1028,0.1654,0.2501,1.6296,0.0288,-0.1461,1.2824
4,14.2830,0.1036,0.2484,1.2368,0.0388,-0.2524,1.3322
5,11.3377,0.2159,0.3896,1.5748,0.0145,-0.1681,3.6544
6,13.1783,0.1713,0.3420,1.2610,-0.0124,-0.2488,1.7642
7,12.3515,0.1952,0.2158,0.9839,-0.1115,-0.2765,0.4709
8,8.7666,0.1799,0.2314,1.0227,0.0325,-0.2505,0.4312
9,7.1279,0.1580,0.2652,1.0016,0.0153,-0.1049,0.6398
